In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utilities

# text processing
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet', '/nltk_data')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# models
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier


[nltk_data] Downloading package wordnet to /nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wo

# We import glove file in order to explore new method based on its word vect representation

In [5]:
"""embed_dict = {}
word_dict = {}
with open("/kaggle/input/glove-100d/glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        word_dict[word] = word
        vector = np.asarray(values[1:], "float32")
        embed_dict[word] = vector"""


'embed_dict = {}\nword_dict = {}\nwith open("/kaggle/input/glove-100d/glove.6B.100d.txt", \'r\', encoding="utf-8") as f:\n    for line in f:\n        values = line.split()\n        word = values[0]\n        word_dict[word] = word\n        vector = np.asarray(values[1:], "float32")\n        embed_dict[word] = vector'

# Function to clean the text

In [23]:
def remove_stop_word(text):
    """
        remove stop word in text, 
        such as the, as, is...
        
        Args : 
            text (str) :
                the text we want to remove stopword
                
        returns :
            text without stop word
    """
    
    STOPWORDS = set(stopwords.words('english'))

    return " ".join([word for word in text.split() if word not in STOPWORDS])
    

def del_pattern(pattern, text):
    """
        Del specific pattern in text
        
        Args :
            pattern (str) : 
                the pattern we want to remove
            
            text (str) :
                the text we want to clean
        
        Returns :
            text without pattern
    """
    
    return re.sub(pattern, '', text)

def lemmatize(text):
    """
        Lemmatize text : 
        bats -> bat
        cats -> cat
        
        Args :
            text (str) :
                the text we want to lemmatize
                
        returns :
            the text lemmatize
    """
    
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split()])


def del_patterns(patterns, text):
    """
        Del specific patterns in text
        
        Args :
            pattern (list) : 
                the patterns we want to remove
            
            text (str) :
                the text we want to clean
        
        Returns :
            text without patterns
    """
    
    for pattern in patterns:
        text = del_pattern(pattern, text)
    
    return text

def clean_text(text):
    """ 
        The goal og this function is to clean a text by removing stopwords, special caracters...

        Args:
            text (str): 
                text we want to clean

        Returns:
            the text cleaned
    """
    
    text = str(text).lower()
    
    special_char = r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    url = re.compile(r"https?://\S+|www\.\S+")
    html = re.compile(r"<.*?>")    
    num_in_words = r'\d+'
    emoji = re.compile(r"[\U0001F600-\U0001F64F"  # Emoticons
                   "\U0001F300-\U0001F5FF"  # Symbols & pictographs
                   "\U0001F680-\U0001F6FF"  # Transport & map symbols
                   "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                   "\U00002702-\U000027B0"  # Dingbats
                   "\U000024C2-\U0001F251"  # Enclosed characters
                   "]+", 
                  flags=re.UNICODE)
    
    patterns = [rf'[{re.escape(special_char)}]', 
               url,
               html,
               emoji,
               num_in_words]
    
    text = del_patterns(patterns, text)
    
    return lemmatize(text)

In [24]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [25]:
X_test = test["text"].apply(clean_text).tolist()
id_test = test["id"]

X_train = train["text"].apply(clean_text).tolist()
Y_train = train["target"].tolist()

# TFIDF

TF-IDF (Term Frequency-Inverse Document Frequency): this is a widely-used method that calculates a score for each term by combining a value that depends on its frequency of appearance in a text (term frequency) and a second value that depends on its appearance in all texts. The advantages are that the method is simple to implement and therefore effective, filters out common terms by lowering their scores, and highlights important rare terms. However, sensitivity to rare terms can exaggerate the score, semantic relations are not taken into account and the method loses effectiveness on long texts.

In [26]:
tfidf_vectorizer = TfidfVectorizer() 

tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

In [27]:
x_train, x_val, y_train, y_val = train_test_split(tfidf_train_vectors,
                                                  Y_train,
                                                  test_size = 0.3, random_state = 42)
print(x_train.shape)

(5329, 20241)


# Classification

In [28]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

XGB = XGBClassifier()
    

XGB.fit(x_train, y_train)

predictions_xgb = XGB.predict(x_val)

print("Accuracy : ",  accuracy_score(y_val, predictions_xgb))

Accuracy :  0.7692644483362522


# Bagging

In [29]:
from sklearn.ensemble import BaggingClassifier

def bag_model(model, x_train, y_train, x_val, y_val):
    bag = BaggingClassifier(estimator=model,
                            n_estimators=10, 
                            random_state=0).fit(x_train, y_train)

    predictions_bag = bag.predict(x_val)

    print("accuracy bag: ",  accuracy_score(y_val, predictions_bag))

    # Utiliser la validation croisée pour évaluer la performance du modèle
    scores = cross_val_score(bag, x_train, y_train, cv=5, scoring='accuracy')
    print("Cross-validation accuracy scores: ", scores)
    print("Mean cross-validation accuracy: ", scores.mean())
    return bag

In [13]:
bag_xgb = bag_model(XGBClassifier(random_state=42), x_train, y_train, x_val, y_val)

accuracy bag:  0.7670753064798599
Cross-validation accuracy scores:  [0.76641651 0.7804878  0.76172608 0.75984991 0.76901408]
Mean cross-validation accuracy:  0.7674988769389318


In [ ]:
bag_cat = bag_model(CatBoostClassifier(random_state=42, verbose=0), x_train, y_train, x_val, y_val)

# Voting

In [15]:
xgb = XGBClassifier(random_state=42)
cat = CatBoostClassifier(random_state=42, verbose=0)
lgb = LGBMClassifier(random_state = 42, verbose=-1)

def voting_models(models, x_train, y_train, x_val, y_val):
    
    voting = VotingClassifier(estimators=models,
                                    voting='soft')

    voting.fit(x_train, y_train)

    predictions_boost = voting.predict(x_val)

    print("accuracy boost: ",  accuracy_score(y_val, predictions_boost))

    # Utiliser la validation croisée pour évaluer la performance du modèle
    scores = cross_val_score(voting, x_train, y_train, cv=5, scoring='accuracy')
    print("Cross-validation accuracy scores: ", scores)
    print("Mean cross-validation accuracy: ", scores.mean())
    
    return voting

models = [('xgb', xgb), ('cat', cat), ('lgb', lgb)]

vot = voting_models(models, x_train, y_train, x_val, y_val)

accuracy boost:  0.7732049036777583
Cross-validation accuracy scores:  [0.77204503 0.78705441 0.76454034 0.76078799 0.76525822]
Mean cross-validation accuracy:  0.7699371966634077


# Submission

In [16]:
def submission(model, test, Id):
    pred = model.predict(test)

    submission = pd.DataFrame({'id': Id,
                               'target': pred})

    # Save submission to a CSV file
    submission.to_csv('submission.csv', index=False)
    
submission(vot, tfidf_test_vectors, id_test)